In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from io import StringIO

In [2]:
data = pd.read_csv('data/classic-rock-raw-data.csv', chunksize=1000)

In [3]:
chunk_list = []  # List to hold the chunks as DataFrames
for chunk in pd.read_csv('data/classic-rock-raw-data.csv', chunksize=1000):
    chunk_list.append(chunk)

# Concatenate all chunks
song_plays = pd.concat(chunk_list, axis=0)
song_plays

,SONG RAW,Song Clean,ARTIST RAW,ARTIST CLEAN,CALLSIGN,TIME,UNIQUE_ID,COMBINED,First?
0,Caught Up In (live),Caught Up in You,.38 Special,.38 Special,KGLK,1402943314,KGLK1536,Caught Up in You by .38 Special,1
1,Caught Up In You,Caught Up in You,.38 Special,.38 Special,KGB,1403398735,KGB0260,Caught Up in You by .38 Special,0
2,Caught Up In You,Caught Up in You,.38 Special,.38 Special,KGB,1403243924,KGB0703,Caught Up in You by .38 Special,0
3,Caught Up in You,Caught Up in You,.38 Special,.38 Special,KGLK,1403470732,KGLK0036,Caught Up in You by .38 Special,0
4,Caught Up in You,Caught Up in You,.38 Special,.38 Special,KGLK,1403380737,KGLK0312,Caught Up in You by .38 Special,0
...,...,...,...,...,...,...,...,...,...
79601,In My Room,NaN,NaN,NaN,KGLK,1403256501,KGLK0698,by,0
79602,Lucky man,NaN,ELP,NaN,KGON,1403409535,KGON0139,by,0
79603,Mrs. Robinson,NaN,NaN,NaN,KGLK,1403083686,KGLK1227,by,0
79604,So Far Away,NaN,NaN,NaN,KGLK,1402991944,KGLK1386,by,0


In [4]:
# Drop unnecessary columns
song_plays.drop('TIME', axis=1, inplace=True)
song_plays.drop('First?', axis=1, inplace=True)
song_plays.drop('SONG RAW', axis=1, inplace=True)
song_plays.drop('ARTIST RAW', axis=1, inplace=True)
song_plays.rename(columns={'Song Clean': 'Song', 'ARTIST CLEAN': 'Artist', 'CALLSIGN': 'Callsign', 'COMBINED': 'Combined'}, inplace=True)

In [5]:
song_plays.isna().sum()

Song         26
Artist        8
Callsign      0
UNIQUE_ID     0
Combined      0
dtype: int64

### In a DataFrame containing nearly 80,000 rows, there are ~30 containing a missing value. We will drop those rows.

In [6]:
song_plays = song_plays.dropna()

### As we see below, this dataset really only has three meaningful columns: song title, artist, and station callsign. However, we can merge with other datasources to expand the number of variables.

In [7]:
song_plays

,Song,Artist,Callsign,UNIQUE_ID,Combined
0,Caught Up in You,.38 Special,KGLK,KGLK1536,Caught Up in You by .38 Special
1,Caught Up in You,.38 Special,KGB,KGB0260,Caught Up in You by .38 Special
2,Caught Up in You,.38 Special,KGB,KGB0703,Caught Up in You by .38 Special
3,Caught Up in You,.38 Special,KGLK,KGLK0036,Caught Up in You by .38 Special
4,Caught Up in You,.38 Special,KGLK,KGLK0312,Caught Up in You by .38 Special
...,...,...,...,...,...
79593,Tush,ZZ Top,WZLX,WZLX0773,Tush by ZZ Top
79594,Tush,ZZ Top,WZLX,WZLX1158,Tush by ZZ Top
79595,TV Dinners,ZZ Top,KZOK,KZOK0830,TV Dinners by ZZ Top
79596,WAITIN' FOR THE BUS/JESUS JUST LEFT CHICAGO,ZZ Top,KCFX,KCFX0650,WAITIN' FOR THE BUS/JESUS JUST LEFT CHICAGO by...


### There are 25 stations represented in the dataset. What can we do with this information?

In [8]:
callsigns = song_plays['Callsign'].unique()
print(callsigns.size)
print(callsigns)

25
['KGLK' 'KGB' 'KRFX' 'KSHE' 'KZEP' 'KZOK' 'KZPS' 'WAXQ' 'WBGG' 'WBIG'
 'WCSX' 'WDVE' 'WMGK' 'WNCX' 'WRFX' 'WXGL' 'KCFX' 'KSEG' 'KSLX' 'WLUP'
 'WOFX' 'KLOS' 'KUFX' 'KGON' 'WZLX']


### Wikipedia has tables containing information about every FM radio station in the United States. Unfortunately, it is not very accessable and is split over 16 pages.

In [9]:
wiki_url = ['https://en.wikipedia.org/wiki/List_of_FM_radio_stations_in_the_United_States_by_call_sign_(initial_letters_KA%E2%80%93KC)',
            'https://en.wikipedia.org/wiki/List_of_FM_radio_stations_in_the_United_States_by_call_sign_(initial_letters_KD%E2%80%93KF)',
            'https://en.wikipedia.org/wiki/List_of_FM_radio_stations_in_the_United_States_by_call_sign_(initial_letters_KG%E2%80%93KJ)',
            'https://en.wikipedia.org/wiki/List_of_FM_radio_stations_in_the_United_States_by_call_sign_(initial_letters_KK%E2%80%93KM)',
            'https://en.wikipedia.org/wiki/List_of_FM_radio_stations_in_the_United_States_by_call_sign_(initial_letters_KN%E2%80%93KP)',
            'https://en.wikipedia.org/wiki/List_of_FM_radio_stations_in_the_United_States_by_call_sign_(initial_letters_KQ%E2%80%93KS)',
            'https://en.wikipedia.org/wiki/List_of_FM_radio_stations_in_the_United_States_by_call_sign_(initial_letters_KT%E2%80%93KV)',
            'https://en.wikipedia.org/wiki/List_of_FM_radio_stations_in_the_United_States_by_call_sign_(initial_letters_KW%E2%80%93KZ)',
            'https://en.wikipedia.org/wiki/List_of_FM_radio_stations_in_the_United_States_by_call_sign_(initial_letters_WA%E2%80%93WC)',
            'https://en.wikipedia.org/wiki/List_of_FM_radio_stations_in_the_United_States_by_call_sign_(initial_letters_WD%E2%80%93WF)',
            'https://en.wikipedia.org/wiki/List_of_FM_radio_stations_in_the_United_States_by_call_sign_(initial_letters_WG%E2%80%93WJ)',
            'https://en.wikipedia.org/wiki/List_of_FM_radio_stations_in_the_United_States_by_call_sign_(initial_letters_WK%E2%80%93WM)',
            'https://en.wikipedia.org/wiki/List_of_FM_radio_stations_in_the_United_States_by_call_sign_(initial_letters_WN%E2%80%93WP)',
            'https://en.wikipedia.org/wiki/List_of_FM_radio_stations_in_the_United_States_by_call_sign_(initial_letters_WQ%E2%80%93WS)',
            'https://en.wikipedia.org/wiki/List_of_FM_radio_stations_in_the_United_States_by_call_sign_(initial_letters_WT%E2%80%93WV)',
            'https://en.wikipedia.org/wiki/List_of_FM_radio_stations_in_the_United_States_by_call_sign_(initial_letters_WW%E2%80%93WZ)']

In [10]:
# all_stations = pd.DataFrame()
# for page in wiki_url:
#     response = requests.get(page)
#     soup = BeautifulSoup(response.text, 'html.parser')
#     station_list = soup.find('table', attrs={'class': "wikitable sortable"})
#     html_string = str(station_list)
#     html_string_io = StringIO(html_string)
#     station_chunk = pd.read_html(html_string_io)[0]
#     all_stations = pd.concat([all_stations, station_chunk], ignore_index=True)

In [11]:
all_stations = pd.DataFrame()
for page in wiki_url:
    response = requests.get(page)
    soup = BeautifulSoup(response.text, 'html.parser')
    station_lists = soup.find_all('table', attrs={'class': "wikitable sortable"})  # Use find_all to get all tables

    for station_list in station_lists:
        html_string = str(station_list)
        html_string_io = StringIO(html_string)
        station_chunk = pd.read_html(html_string_io)[0]
        all_stations = pd.concat([all_stations, station_chunk], ignore_index=True)

In [12]:
all_stations = all_stations.drop('Frequency', axis=1)

In [13]:
all_stations[all_stations['City of license'].str.startswith('Kansas')]

,Callsign,City of license
333,KBEQ-FM,"Kansas City, Missouri"
652,KCKC,"Kansas City, Missouri"
761,KCUR-FM,"Kansas City, Missouri"
1167,KFKF-FM,"Kansas City, Kansas"
1882,KJNW,"Kansas City, Missouri"
1991,KKFI,"Kansas City, Missouri"
2476,KMBZ-FM,"Kansas City, Kansas"
2683,KMXV,"Kansas City, Missouri"
3211,KPRS,"Kansas City, Missouri"
3462,KRBZ,"Kansas City, Missouri"


In [14]:
# Remove '-FM' found in some callsigns
all_stations['Callsign'] = all_stations['Callsign'].apply(lambda x: x.split('-')[0])

### A scraped, concatenated, and cleaned DataFrame containing the location of each station. May split the location into city and state columns at a later time. TBD.

In [15]:
all_stations

,Callsign,City of license
0,KAAC,"Utqiavik, Alaska"
1,KAAI,"Palisade, Colorado"
2,KAAK,"Great Falls, Montana"
3,KAAN,"Bethany, Missouri"
4,KAAQ,"Alliance, Nebraska"
...,...,...
10929,WZZR,"West Palm Beach, Florida"
10930,WZZT,"Morrison, Illinois"
10931,WZZU,"Lynchburg, Virginia"
10932,WZZY,"Winchester, Indiana"


In [16]:
song_plays = pd.merge(song_plays, all_stations, how='left', on='Callsign')

In [17]:
cities = song_plays['City of license'].unique()
print(cities.size)
print(cities)

25
['Lake Jackson, Texas' 'San Diego, California' 'Denver, Colorado'
 'Crestwood, Missouri' 'San Antonio, Texas' 'Seattle, Washington'
 'Dallas, Texas' 'New York City' 'Fort Lauderdale, Florida'
 'Washington, D.C.' 'Birmingham, Michigan' 'Pittsburgh, Pennsylvania'
 'Philadelphia' 'Cleveland, Ohio' 'Kannapolis, North Carolina'
 'St. Petersburg, Florida' 'Harrisonville, Missouri'
 'Sacramento, California' 'Scottsdale, Arizona' 'Cambridge, Minnesota'
 'Cincinnati' 'Los Angeles' 'San Jose, California' 'Portland, Oregon'
 'Boston, Massachusetts']


In [18]:
song_plays

,Song,Artist,Callsign,UNIQUE_ID,Combined,City of license
0,Caught Up in You,.38 Special,KGLK,KGLK1536,Caught Up in You by .38 Special,"Lake Jackson, Texas"
1,Caught Up in You,.38 Special,KGB,KGB0260,Caught Up in You by .38 Special,"San Diego, California"
2,Caught Up in You,.38 Special,KGB,KGB0703,Caught Up in You by .38 Special,"San Diego, California"
3,Caught Up in You,.38 Special,KGLK,KGLK0036,Caught Up in You by .38 Special,"Lake Jackson, Texas"
4,Caught Up in You,.38 Special,KGLK,KGLK0312,Caught Up in You by .38 Special,"Lake Jackson, Texas"
...,...,...,...,...,...,...
79575,Tush,ZZ Top,WZLX,WZLX0773,Tush by ZZ Top,"Boston, Massachusetts"
79576,Tush,ZZ Top,WZLX,WZLX1158,Tush by ZZ Top,"Boston, Massachusetts"
79577,TV Dinners,ZZ Top,KZOK,KZOK0830,TV Dinners by ZZ Top,"Seattle, Washington"
79578,WAITIN' FOR THE BUS/JESUS JUST LEFT CHICAGO,ZZ Top,KCFX,KCFX0650,WAITIN' FOR THE BUS/JESUS JUST LEFT CHICAGO by...,"Harrisonville, Missouri"


In [19]:
song_data = pd.read_csv('data/classic-rock-song-list.csv')

In [20]:
song_data

,Song Clean,ARTIST CLEAN,Release Year,COMBINED,First?,Year?,PlayCount,F*G
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1
...,...,...,...,...,...,...,...,...
2224,She Loves My Automobile,ZZ Top,NaN,She Loves My Automobile by ZZ Top,1,0,1,0
2225,Tube Snake Boogie,ZZ Top,1981,Tube Snake Boogie by ZZ Top,1,1,32,32
2226,Tush,ZZ Top,1975,Tush by ZZ Top,1,1,109,109
2227,TV Dinners,ZZ Top,1983,TV Dinners by ZZ Top,1,1,1,1


In [21]:
song_years = song_data.drop(song_data.columns[[0, 1, 4, 5, 6, 7]], axis=1)

In [23]:
song_years.rename(columns={'COMBINED': 'Combined'}, inplace=True)

In [27]:
song_years[song_years['Release Year'].isna()]

,Release Year,Combined
1,NaN,Fantasy Girl by .38 Special
10,NaN,"Baby, Please Don't Go by AC/DC"
13,NaN,CAN'T STOP ROCK'N'ROLL by AC/DC
16,NaN,Girls Got Rhythm by AC/DC
24,NaN,Let's Get It Up by AC/DC
...,...,...
2215,NaN,"I'm Bad, I'm Nationwide by ZZ Top"
2217,NaN,Just Got Paid by ZZ Top
2220,NaN,My Head's In Mississippi by ZZ Top
2221,NaN,Party On The Patio by ZZ Top


In [28]:
song_plays = pd.merge(song_plays, song_years, how='left', on='Combined')

In [30]:
song_plays[song_plays['Release Year'].isna()]

,Song,Artist,Callsign,UNIQUE_ID,Combined,City of license,Release Year
82,Fantasy Girl,.38 Special,WXGL,WXGL0979,Fantasy Girl by .38 Special,"St. Petersburg, Florida",NaN
83,Fantasy Girl,.38 Special,WOFX,WOFX0415,Fantasy Girl by .38 Special,Cincinnati,NaN
84,Fantasy Girl,.38 Special,KGLK,KGLK1535,Fantasy Girl by .38 Special,"Lake Jackson, Texas",NaN
213,"Baby, Please Don't Go",AC/DC,KRFX,KRFX1299,"Baby, Please Don't Go by AC/DC","Denver, Colorado",NaN
317,CAN'T STOP ROCK'N'ROLL,AC/DC,KCFX,KCFX0602,CAN'T STOP ROCK'N'ROLL by AC/DC,"Harrisonville, Missouri",NaN
...,...,...,...,...,...,...,...
79306,Party On The Patio,ZZ Top,WDVE,WDVE1329,Party On The Patio by ZZ Top,"Pittsburgh, Pennsylvania",NaN
79307,Party On The Patio,ZZ Top,WRFX,WRFX1206,Party On The Patio by ZZ Top,"Kannapolis, North Carolina",NaN
79308,Party On The Patio,ZZ Top,WRFX,WRFX1211,Party On The Patio by ZZ Top,"Kannapolis, North Carolina",NaN
79309,Party On The Patio,ZZ Top,WRFX,WRFX1419,Party On The Patio by ZZ Top,"Kannapolis, North Carolina",NaN


### About 25% of the songs are missing the release year (576/2229), but this only represents about 7% of the plays (5592/79580). Not sure the best way to deal with this. 